In [18]:
import pandas as pd
import numpy as np

In [19]:
all2015 = pd.read_csv("Input/2015-all.csv", encoding = 'ISO-8859-1', low_memory=True)
all2015 = all2015.replace(0, np.nan)

C:\Users\guanz\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (13,151,615,1117,1219,1220,1255,1311,1312,2501,2542,2635,2779,3003,3289,3335,3386,3421,4710,5167) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [21]:
all2015 = all2015.dropna(axis=1,how='all')

In [23]:
all2015 = all2015[np.isfinite(all2015['HIOSIssuerID:'])]

In [36]:
all2015 = all2015.rename(columns={"HIOSIssuerID:": "HIOS ID"})

In [48]:
all2015 = all2015.drop_duplicates(subset=['HIOS ID'])

In [37]:
all2015.head()

Unnamed: 0                                CompanyName:  \
0         NaN                                 AvMed, Inc.   
3         3.0  Blue Cross and Blue Shield of Kansas, Inc.   
5         5.0            UnitedHealthcare of Oregon, Inc.   
7         7.0                TRH Health Insurance Company   
9         9.0            Horizon Healthcare of New Jersey   

              GroupAffiliation:  FederalEIN:  A.M.BestNumber:  NAICGroupCode:  \
0                           NaN    592742907          64074.0             NaN   
3                BCBS of KS GRP    480952857          60070.0           430.0   
5              UNITEDHEALTH GRP    930938819          68707.0           707.0   
7  TRH Health Insurance Company    621779831              NaN          4677.0   
9                BCBS OF NJ GRP    222651245          64022.0          1202.0   

   NAICCompanyCode:                          DBA/MarketingName:  HIOS ID  \
0           95263.0                          AvMed Health Plans  19898.0   
3           70729.0  Blue Cross and Blue Shield of Kansas, Inc.  18558.0   
5           95893.0            UnitedHealthcare of Oregon, Inc.  37057.0   
7           89005.0                TRH Health Insurance Company  16348.0   
9           95529.0                              Horizon HMO NJ  13953.0   

  BusinessintheStateof:  \
0               Florida   
3                Kansas   
5                Oregon   
7             Tennessee   
9            New Jersey   

                                                   ...                                                    \
0                                                  ...                                                     
3                                                  ...                                                     
5                                                  ...                                                     
7                                                  ...                                                     
9                                                  ...                                                     

  2.17bTotalfraudrecoveriesthatreducedpaidclaimsinLine2.136StudentHealthINDIVIDUALTotalasof3/31/15  \
0                                                NaN                                                 
3                                                NaN                                                 
5                                                NaN                                                 
7                                                NaN                                                 
9                                                NaN                                                 

  2.17bTotalfraudrecoveriesthatreducedpaidclaimsinLine2.140GovernmentProgramPlansTotalasof12/31/14  \
0                                                NaN                                                 
3                                                NaN                                                 
5                                                NaN                                                 
7                                                NaN                                                 
9                                                NaN                                                 

  2.17bTotalfraudrecoveriesthatreducedpaidclaimsinLine2.141OtherHealthBusinessTotalasof12/31/14  \
0                                                NaN                                              
3                                            11677.0                                              
5                                                NaN                                              
7                                                NaN                                              
9                                                NaN                                              

  2.17bTotalfraudrecoveriesthatreducedpaidclaimsinLine2.142

In [49]:
rira2015 = pd.read_csv("Input/2015-Benefit-Year-RI-RA-Updated-numeric.csv")
rira2015 = rira2015.drop(["Unnamed: 0", "Unnamed: 0.1"],axis=1)

In [63]:
rira2015.head()

HIOS ID      HIOS INPUTTED INSURANCE COMPANY NAME STATE  \
0    11082              Aetna Life Insurance Company    AK   
1    38344  Premera Blue Cross Blue Shield of Alaska    AK   
2    38596                    Time Insurance Company    AK   
3    62637         John Alden Life Insurance Company    AK   
4    73836                    Moda Health Plan, Inc.    AK   

   REINSURANCE PAYMENT AMOUNT (OR NOT ELIGIBLE)  \
0                                          0.00   
1                                   14655165.10   
2                                    1167367.38   
3                                           NaN   
4                                   26305543.39   

   HHS RISK ADJUSTMENT TRANSFER AMOUNT (INDIVIDUAL MARKET, INCLUDING CATASTROPHIC)  \
0                                          -34550.55                                 
1                                        12358455.76                                 
2                                        -2908793.99                                 
3                                                NaN                                 
4                                        -9415111.19                                 

   HHS RISK ADJUSTMENT TRANSFERS AMOUNT (SMALL GROUP MARKET)  
0                                          557129.08          
1                                         1742413.11          
2                                        -1465993.25          
3                                          607207.25          
4                                         -683234.43

In [52]:
merged2015 = rira2015.merge(all2015, on='HIOS ID', how='left')

In [56]:
states = merged2015["STATE"].drop_duplicates()
#print(states)

In [68]:
print("state,individualratio,smallgroupratio,individualtransfers,smallgrouptransfers,individualmm,smallgroupmm")
for state in states:
    temp = merged2015.where(merged2015["STATE"] == state).dropna(how='all')
    a = sum(abs(temp["HHS RISK ADJUSTMENT TRANSFER AMOUNT (INDIVIDUAL MARKET, INCLUDING CATASTROPHIC)"]).convert_objects(convert_numeric=True).dropna())
    b = sum(abs(temp["HHS RISK ADJUSTMENT TRANSFERS AMOUNT (SMALL GROUP MARKET)"]).convert_objects(convert_numeric=True).dropna())
    c = sum(temp["7.4Membermonths2HealthInsuranceINDIVIDUALTotalasof3/31/15"].convert_objects(convert_numeric=True).dropna())
    d = sum(temp["7.4Membermonths7HealthInsuranceSMALLGROUPTotalasof3/31/15"].convert_objects(convert_numeric=True).dropna())
    individual_ratio = a/c
    smallgroup_ratio = b/d
    print(state + "," + str(individual_ratio) + "," + str(smallgroup_ratio) + "," +
          str(a) + "," + str(b) + "," + str(c) + "," + str(d))

state,individualratio,smallgroupratio,individualtransfers,smallgrouptransfers,individualmm,smallgroupmm
AK,75.9058290493,17.8564813495,24716911.49,5813498.92,325626.0,325568.0
AL,9.41952716298,1.36972222751,26984194.82,3882891.31,2864708.0,2834802.0


C:\Users\guanz\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  after removing the cwd from sys.path.
C:\Users\guanz\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  """
C:\Users\guanz\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  
C:\Users\guanz\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  import sys


AR,7.6272759049,2.52594034274,30195874.28,3852172.69,3958933.0,1525045.0
AZ,35.498774775,13.7372186702,137427755.74,29347438.42,3871338.0,2136345.0
CA,23.2808723061,16.0037170988,628633885.51,327333099.93,27002162.0,20453567.0
CO,29.5011554816,7.67982376148,98259941.08,19113875.61,3330715.0,2488843.0
CT,34.90164113,28.9389420989,71536742.07,61008064.87,2049667.0,2108165.0
DC,30.5561524334,28.9875967766,7021162.15,31090559.96,229779.0,1072547.0
DE,125.635002004,42.3622295363,5641262.86,4794387.69,44902.0,113176.0
FL,57.840819498,6.30009055787,1177008909.25,50019972.19,20349105.0,7939564.0
GA,28.2658776103,5.86756297927,204697669.76,27926448.9,7241865.0,4759463.0
HI,57.7755799186,10.2461004888,30934201.0,15970812.0,535420.0,1558721.0
IA,14.9556015642,5.95307387947,34340887.8,11010364.92,2296189.0,1849526.0
ID,13.6834227733,5.98324833186,23193278.45,5556816.24,1694991.0,928729.0
IL,8.77254438625,5.95276064256,67848191.71,36918104.78,7734152.0,6201846.0
IN,39.3838130149,6.3453533349,110276